# Imports

In [1]:
import pandas as pd 
import numpy as np
import pickle

from PIL import Image

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Reading csv and spliting data

In [2]:
df_train = pd.read_csv('../input/mnist-in-csv/mnist_train.csv')
df_train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_train = df_train.reindex(np.random.permutation(df_train.index))

In [4]:
# df_train = df_train.drop(df_train.index[5000:]) # Don't run more than once

In [5]:
df_train.shape

(60000, 785)

In [6]:
X_train = df_train.drop(columns = ['label'])
X_train

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
46470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y_train = df_train['label']
y_train

46470    1
13520    4
18828    9
20072    3
32010    2
        ..
48695    0
57203    1
17264    5
1957     6
43638    5
Name: label, Length: 60000, dtype: int64

In [8]:
df_test = pd.read_csv('../input/mnist-in-csv/mnist_test.csv')
df_test

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_test = df_test.reindex(np.random.permutation(df_test.index))

In [10]:
# df_test = df_test.drop(df_test.index[999:9999]) # Don't run more than once

In [11]:
df_test.shape

(10000, 785)

In [12]:
X_test = df_test.drop(columns = ['label'])
X_test

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
2137,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2092,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2651,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4955,0,0,0,0,0,0,0,0,0,0,...,67,0,0,0,0,0,0,0,0,0
2936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_test = df_test['label']
y_test

2137    1
2092    2
9427    5
2651    6
7600    1
       ..
9251    6
4955    7
2936    4
7223    1
5316    1
Name: label, Length: 10000, dtype: int64

# Random Forest Classifier with Hyperparameter Optimization

In [14]:
rf_classifier = RandomForestClassifier()

In [15]:
param_space = {'n_estimators' : [10,50,80,90,100,1000], 
               'max_depth' : [10,70,80,90,100,1000], 
               'min_samples_split' : [10,50,60,70,80,90,1000], 
               'min_samples_leaf' : [1,10,60,80,90,100,1000], 
               'max_features' : [2,4,6,8]}

Already used cell below to find best params

In [16]:
# rf_classifier_cv = RandomizedSearchCV(rf_classifier, param_space, n_iter=15, scoring = 'neg_root_mean_squared_error', cv = 5 )
# search = rf_classifier_cv.fit(X_train, y_train)

search.best_params_: max_depth=10, max_features=8, min_samples_leaf=1, min_samples_split=70, n_estimators=1000

In [17]:
rf_classifier = RandomForestClassifier(max_depth=10, max_features=8, min_samples_leaf=1, min_samples_split=70, n_estimators=1000)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features=8, min_samples_split=70,
                       n_estimators=1000)

In [18]:
rf_predictions = rf_classifier.predict(X_test)

In [19]:
mean_absolute_error(y_test, rf_predictions)

0.226

In [20]:
accuracy_score(y_test, rf_predictions)*100 # %

94.02000000000001

In [21]:
precision_score(y_test, rf_predictions, average='macro')*100 # %

94.06728041460056

In [22]:
recall_score(y_test, rf_predictions, average='macro')*100 # %

93.94081199569133

# Saving model


In [24]:
filename = 'final_rf_model.pkl'
pickle.dump(rf_classifier, open(filename, 'wb'))

# Testing with my own digits

In [26]:
img1 = Image.open("../input/shapes/0.png").convert(mode="1")
array1 = np.array(img1.getdata())

In [40]:
img2 = Image.open("../input/shapes/1.png").convert(mode="1")
array2 = np.array(img2.getdata())

In [42]:
df_myimages = pd.DataFrame(np.array([array1,array2]))

In [43]:
df_myimages

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Predict with my images**

In [44]:
rf_classifier.predict(df_myimages)

array([0, 1])

# Opening and using saved model

In [ ]:
loaded_model = pickle.load(open(filename,'rb'))

In [46]:
loaded_model.predict(df_myimages)

array([0, 1])